In [1]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

--2025-02-08 05:01:22--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26622 (26K) [text/plain]
Saving to: ‘edgar_allan_poe.txt’

edgar_allan_poe.txt 100%[===================>]  26.00K  --.-KB/s    in 0.002s  

2025-02-08 05:01:23 (16.6 MB/s) - ‘edgar_allan_poe.txt’ saved [26622/26622]

--2025-02-08 05:01:23--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP re

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [3]:
input_files = [
  'edgar_allan_poe.txt',
  'robert_frost.txt',
]

In [4]:
!head edgar_allan_poe.txt

LO! Death hath rear'd himself a throne
In a strange city, all alone,
Far down within the dim west
Where the good, and the bad, and the worst, and the best,
Have gone to their eternal rest.
 
There shrines, and palaces, and towers
Are not like any thing of ours
Oh no! O no! ours never loom
To heaven with that ungodly gloom!


In [5]:
!head robert_frost.txt

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth; 

Then took the other, as just as fair,
And having perhaps the better claim
Because it was grassy and wanted wear,
Though as for that the passing there


In [6]:
# collect data into lists
input_texts = []
labels = []

for label, f in enumerate(input_files):
  print(f"{f} corresponds to label {label}")

  for line in open(f):
    line = line.rstrip().lower()
    if line:
      # remove punctuation
      line = line.translate(str.maketrans('', '', string.punctuation))

      input_texts.append(line)
      labels.append(label)

edgar_allan_poe.txt corresponds to label 0
robert_frost.txt corresponds to label 1


In [7]:
train_text, test_text, Ytrain, Ytest = train_test_split(input_texts, labels)

In [8]:
len(Ytrain), len(Ytest)

(1615, 539)

In [9]:
train_text[:5]

['with the weight of an age of snows',
 'and built a stone baptismal font outdoors',
 'hed like to take me there and show it to me',
 'where the good and the bad and the worst and the best',
 'therefore thou art not wrong']

In [10]:
Ytrain[:5]

[0, 1, 1, 0, 0]

In [11]:
idx = 1
word2idx = {'<unk>': 0}

In [12]:
# populate word2idx
for text in train_text:
    tokens = text.split()
    for token in tokens:
      if token not in word2idx:
        word2idx[token] = idx
        idx += 1

In [13]:
word2idx

{'<unk>': 0,
 'with': 1,
 'the': 2,
 'weight': 3,
 'of': 4,
 'an': 5,
 'age': 6,
 'snows': 7,
 'and': 8,
 'built': 9,
 'a': 10,
 'stone': 11,
 'baptismal': 12,
 'font': 13,
 'outdoors': 14,
 'hed': 15,
 'like': 16,
 'to': 17,
 'take': 18,
 'me': 19,
 'there': 20,
 'show': 21,
 'it': 22,
 'where': 23,
 'good': 24,
 'bad': 25,
 'worst': 26,
 'best': 27,
 'therefore': 28,
 'thou': 29,
 'art': 30,
 'not': 31,
 'wrong': 32,
 'thats': 33,
 'his': 34,
 'mistake': 35,
 'dear': 36,
 'knows': 37,
 'my': 38,
 'interest': 39,
 'or': 40,
 'creaking': 41,
 'buggy': 42,
 'load': 43,
 'grain': 44,
 'though': 45,
 'what': 46,
 'you': 47,
 'can': 48,
 'be': 49,
 'anyone': 50,
 'had': 51,
 'been': 52,
 'too': 53,
 'short': 54,
 'sell': 55,
 'give': 56,
 'away': 57,
 'her': 58,
 'early': 59,
 'leafs': 60,
 'flower': 61,
 'o': 62,
 'i': 63,
 'care': 64,
 'that': 65,
 'earthly': 66,
 'lot': 67,
 'its': 68,
 'old': 69,
 'captains': 70,
 'dark': 71,
 'fate': 72,
 'end': 73,
 'far': 74,
 'from': 75,
 'going': 

In [14]:
len(word2idx)

2527

In [15]:
# convert data into integer format
train_text_int = []
test_text_int = []

for text in train_text:
  tokens = text.split()
  line_as_int = [word2idx[token] for token in tokens]
  train_text_int.append(line_as_int)

for text in test_text:
  tokens = text.split()
  line_as_int = [word2idx.get(token, 0) for token in tokens]
  test_text_int.append(line_as_int)

In [16]:
train_text_int[100:105]

[[363, 75, 2, 364, 365, 4, 2, 366],
 [2, 367, 240, 368, 2, 369],
 [68, 370, 371],
 [2, 372, 373],
 [118, 262, 10, 374, 357, 375]]

In [17]:
# initialize A and pi matrices - for both classes
V = len(word2idx)

A0 = np.ones((V, V))
pi0 = np.ones(V)

A1 = np.ones((V, V))
pi1 = np.ones(V)

In [18]:
# compute counts for A and pi
def compute_counts(text_as_int, A, pi):
  for tokens in text_as_int:
    last_idx = None
    for idx in tokens:
      if last_idx is None:
        # it's the first word in a sentence
        pi[idx] += 1
      else:
        # the last word exists, so count a transition
        A[last_idx, idx] += 1

      # update last idx
      last_idx = idx


compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 0], A0, pi0)
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 1], A1, pi1)

In [19]:
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

In [20]:
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [21]:
count0 = sum(y == 0 for y in Ytrain)
count1 = sum(y == 1 for y in Ytrain)
total = len(Ytrain)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, p1

(0.3318885448916409, 0.6681114551083591)

In [22]:
# build a classifier
class Classifier:
  def __init__(self, logAs, logpis, logpriors):
    self.logAs = logAs
    self.logpis = logpis
    self.logpriors = logpriors
    self.K = len(logpriors) # number of classes

  def _compute_log_likelihood(self, input_, class_):
    logA = self.logAs[class_]
    logpi = self.logpis[class_]

    last_idx = None
    logprob = 0
    for idx in input_:
      if last_idx is None:
        # it's the first token
        logprob += logpi[idx]
      else:
        logprob += logA[last_idx, idx]

      # update last_idx
      last_idx = idx

    return logprob

  def predict(self, inputs):
    predictions = np.zeros(len(inputs))
    for i, input_ in enumerate(inputs):
      posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] \
             for c in range(self.K)]
      pred = np.argmax(posteriors)
      predictions[i] = pred
    return predictions

In [23]:
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [24]:
Ptrain = clf.predict(train_text_int)
print(f"Train acc: {np.mean(Ptrain == Ytrain)}")

Train acc: 0.9962848297213622


In [25]:
Ptest = clf.predict(test_text_int)
print(f"Test acc: {np.mean(Ptest == Ytest)}")

Test acc: 0.8237476808905381


In [26]:
from sklearn.metrics import confusion_matrix, f1_score

In [27]:
cm = confusion_matrix(Ytrain, Ptrain)
cm

array([[ 530,    6],
       [   0, 1079]])

In [28]:
cm_test = confusion_matrix(Ytest, Ptest)
cm_test

array([[ 94,  88],
       [  7, 350]])

In [29]:
f1_score(Ytrain, Ptrain)

0.9972273567467652

In [30]:
f1_score(Ytest, Ptest)

0.8805031446540881